<a href="https://colab.research.google.com/github/rohanath123/Latent-Dirichlet-Allocation/blob/master/LDA_SPOOKYAUTHORS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np 
import re 
from pprint import pprint

In [0]:
import gensim 
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

In [0]:
import spacy
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
df = pd.read_csv('/content/drive/My Drive/Datasets/Spooky Authors/train.csv')

In [0]:
df = df.drop(['id', 'author'],axis = 1)

In [0]:
data = df.values.tolist()

In [0]:
'''
1. TOKENIZE
2. REMOVE STOPWORDS
3. CREATE BIGRAMS AND TRIGRAMS 
4. LEMMATIZE
5. ID2WORD
6. CORPUS (I.E. BOW)
7. MODEL
'''

'\n1. TOKENIZE\n2. REMOVE STOPWORDS\n3. CREATE BIGRAMS AND TRIGRAMS \n4. LEMMATIZE\n5. ID2WORD\n6. CORPUS (I.E. BOW)\n7. MODEL\n'

TOKENIZATION

In [0]:
def sent_to_words(sentences):
  for sent in sentences:
    yield(gensim.utils.simple_preprocess(str(sent), deacc = True))

In [0]:
data_words = list(sent_to_words(data))

REMOVE STOPWORDS

In [0]:
stop_words = stopwords.words('english')

In [0]:
def remove_stopwords(texts):
  return([word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts)

MAKE BIGRAMS AND TRIGRAMS FUNTIONS

In [0]:
bigrams = gensim.models.Phrases(data_words, min_count = 5, threshold = 100)
trigrams = gensim.models.Phrases(bigrams[data_words], threshold = 100)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
bigram_mod = gensim.models.phrases.Phraser(bigrams)
trigram_mod = gensim.models.phrases.Phraser(trigrams)

In [0]:
def make_bigrams(texts):
  return([bigram_mod[doc] for doc in texts])
def make_trigrams(texts):
  return([trigram_mod[doc] for doc in texts])

LEMMATIZATION

In [0]:
def lemma(texts, allowed_postags = ['NOUN', 'ADJ', 'ADV', 'VERB']):
  texts_out = []
  for sent in texts:
    doc = nlp(" ".join(sent))
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  return texts_out

ACTUAL DATA CLEANING

In [0]:
data_words_nonstops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nonstops)

In [0]:
nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])

In [0]:
data_lem = lemma(data_words_bigrams)

In [0]:
id2word = corpora.Dictionary(data_lem)

In [0]:
texts = data_lem

In [0]:
corpus = [id2word.doc2bow(text) for text in texts]

In [0]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, num_topics = 15, id2word = id2word, passes = 10, alpha = 'auto', per_word_topics = True)

In [0]:
pprint(lda_model.print_topics())

[(0,
  '0.012*"thing" + 0.010*"other" + 0.009*"understand" + 0.009*"therefore" + '
  '0.007*"earth" + 0.007*"always" + 0.006*"sympathy" + 0.006*"easily" + '
  '0.006*"certain" + 0.005*"put"'),
 (1,
  '0.027*"eye" + 0.010*"horror" + 0.010*"seem" + 0.008*"sound" + '
  '0.007*"scream" + 0.007*"extreme" + 0.006*"folk" + 0.006*"rush" + '
  '0.006*"bottom" + 0.006*"lip"'),
 (2,
  '0.014*"true" + 0.012*"nature" + 0.009*"fail" + 0.008*"singular" + '
  '0.008*"west" + 0.007*"employ" + 0.007*"concern" + 0.006*"right" + '
  '0.006*"ordinary" + 0.006*"similar"'),
 (3,
  '0.021*"open" + 0.021*"door" + 0.018*"room" + 0.010*"close" + 0.010*"fall" + '
  '0.009*"throw" + 0.008*"chamber" + 0.008*"answer" + 0.008*"window" + '
  '0.008*"perceive"'),
 (4,
  '0.010*"see" + 0.009*"black" + 0.009*"thing" + 0.008*"light" + 0.008*"stone" '
  '+ 0.008*"face" + 0.008*"terrible" + 0.007*"eye" + 0.006*"look" + '
  '0.006*"shape"'),
 (5,
  '0.035*"house" + 0.014*"town" + 0.013*"ancient" + 0.012*"old" + '
  '0.006*"s

In [0]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 3.3MB/s 
     |████████████████████████████████| 552kB 20.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=1c00e88c8a50cd2551feb116d517843006cfc0d5a88ab0316fc77e5865c031cb
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32040 sha256=3c8c109a3a93f900c89a59d2e9daba68071091b0c2260b7649b284c666c3eeb0
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
from gensim.models import CoherenceModel

In [0]:
coh = CoherenceModel(model = lda_model, texts = data_lem, dictionary = id2word)

In [0]:
coh.get_coherence_per_topic()

[0.3613336108748617,
 0.4257007555143992,
 0.28465105376925015,
 0.33600424392114403,
 0.44309082926459614,
 0.3578391524880143,
 0.2572543325170049,
 0.5043776994209135,
 0.27042273948036477,
 0.2763773119589108,
 0.3146418976127136,
 0.41687147286627246,
 0.23942788298383552,
 0.2271363133275265,
 0.40065010203608453]